In [7]:
import sys
import os
import os.path
import fnmatch
from lxml import etree
from collections import OrderedDict
import csv
import IPython
import argparse
import re

In [8]:
nums=['I ','V ','X ','C ','L ','I.','V.','X.','C.','L.','1','2','3','4','5','6','7','8','9','0']

In [2]:
print('This software generates a csv with basic statistics on a selected corpus \nusage: --dir path/to/your/source/dir --csv path/to/your/target/directory/for/csv')
parser = argparse.ArgumentParser()
parser.add_argument('--dir', help= '/your/directory/to/tagged/files/')
parser.add_argument('--csv', help= '/your/directory/to/your/csv/file/')
args = parser.parse_args()

This software generates a csv with basic statistics on a selected corpus 
usage: --dir path/to/your/source/dir --csv path/to/your/target/directory/for/csv


usage: ipykernel_launcher.py [-h] [--dir DIR] [--csv CSV]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1000/jupyter/kernel-649c50ff-c59c-4aa1-90d3-cf51bdbbd106.json


SystemExit: 2

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
path_to_folder=args.dir
files_list=fnmatch.filter(os.listdir(path_to_folder), '*.xml')

NameError: name 'args' is not defined

In [9]:
argsdir='/home/odysseus/Bureau/ANR/corpus/tagged_rest/'
argscsv='/home/odysseus/Bureau/ANR/code/global_array/'
files_list=fnmatch.filter(os.listdir(argsdir), '*.xml')

In [10]:
count_header=0
with open(argscsv+'rest.csv', 'a') as f:
    for file in files_list:
        dic_stats=OrderedDict()
        tmpFile=file.replace("/",":")
        full_path=argsdir+tmpFile
        if os.path.isfile(full_path):
            tree=etree.parse(full_path)
            dic_stats['ref']=file
            dic_stats['title']=re.sub(u'\n','',tree.find(".//title").text).replace("     ","")
            dic_stats['author']=tree.find(".//author").attrib['name']
            dic_stats['date']=tree.find(".//date").attrib['when']
            #dic_stats['genre']=tree.find(".//term").text
            dic_stats['canon_degree']="empty"
            dic_stats['word_count']=len(tree.findall(".//word"))
            dic_stats['book_count']=len(tree.findall(".//div[@type='book']"))
            dic_stats['part_count']=len(tree.findall(".//div[@type='part']"))
            dic_stats['chapter_count']=len(tree.findall(".//div[@type='chapter']"))
            dic_stats['paragraph_count']=len(tree.findall(".//p"))
            dic_stats['sentence_count']=len(tree.findall(".//word[@postag='PUNsent']"))
            dic_stats['verb_count']=len(tree.findall(".//word[@postag='VERB']"))
            titles_book=0
            num_book=0
            for element in tree.findall(".//div[@type='book'][@title]"):
                if len(element.attrib["title"])>3:
                    titles_book+=1
                if any(num in element.attrib["title"] for num in nums):
                    num_book+=1
            titles_part=0
            num_part=0
            for element in tree.findall(".//div[@type='part'][@title]"):
                if len(element.attrib["title"])>3:
                    titles_part+=1
                if any(num in element.attrib["title"] for num in nums):
                    num_part+=1
            titles_chap=0
            num_chap=0
            for element in tree.findall(".//div[@type='chapter'][@title]"):
                if len(element.attrib["title"])>3:
                    titles_chap+=1
                if any(num in element.attrib["title"] for num in nums):
                    num_chap+=1
            dic_stats['titled_book_count']=titles_book
            dic_stats['titled_part_count']=titles_part
            dic_stats['titled_chapter_count']=titles_chap
            dic_stats['numbered_book_count']=num_book
            dic_stats['numbered_part_count']=num_part
            dic_stats['numbered_chapter_count']=num_chap
            headers=list(dic_stats.keys())
            writer = csv.DictWriter(f, delimiter=',', lineterminator='\n',fieldnames=headers)
            if (count_header==0):
                writer.writeheader()
                count_header += 1
            writer2 = csv.writer(f)
            writer2.writerow(list(dic_stats.values()))
        print('File done : '+file)

File done : 1858_Segur-comtesse-de_Les-Malheurs-de-Sophie.xml
File done : 1850_Dumas-Alexandre_Le-Vicomte-de-Bragelonne.xml
File done : 2014_Volodine-Antoine_Terminus-radieux.xml
File done : 1909_Boylesve-Rene_La-jeune-fille-bien-elevee.xml
File done : 1872_Zola-Emile_La-curee.xml
File done : 1861_Erckmann-Chatrian_Confidences-d-un-joueur-de-clarinette.xml
File done : 1920_Toulet-Paul-Jean_La-jeune-fille-verte-roman.xml
File done : 1961_Troyat-Henri_La-gloire-des-vaincus.xml
File done : 1888_Malot-Hector_Conscience.xml
File done : 1944_Aragon-Louis_Aurelien.xml
File done : 1951_Rebatet-Lucien_Les-Deux-Etendards.xml
File done : 1991_Bauchau-Henry_Diotime-et-les-lions.xml
File done : 1847_Collin-de-Plancy-Jacques-Albin-Simon-(1794-1881)_Legende-du-Juif-Errant.xml
File done : 1861_Rocca-Maria-della-(nee-Embden-Heine-pseud-Camille-Henry-Princesse)_Le-Roman-d-une-femme-laide.xml
File done : 1977_Ernaux-Annie_Ce-qu-ils-disent-ou-rien.xml
File done : 1843_Doutre-Joseph_Les-fiances-de-1812.xml

File done : 1858_Segur-comtesse-de_Les-Petites-Filles-Modeles.xml
File done : 1869_Stolz-Madame-de_La-Maison-roulante.xml
File done : 1894_Louÿs-Pierre_Les-chansons-de-Bilitis.xml
File done : 1961_San-Antonio_Ne-mangez-pas-la-consigne.xml
File done : 1930_Malraux-Andre_La-voie-royale.xml
File done : 1943_Saint-Exupery-Antoine-de_Le-petit-prince.xml
File done : 1973_Vautrin-Jean_a-Bulletins-Rouges.xml
File done : 1910_Roussel-Raymond_Impressions-d-Afrique.xml
File done : 1922_Rolland-Romain_L-ame-enchantee-Annette-et-Sylvie-Volume-1.xml
File done : 1920_Zevaco-Michel_Le-Pre-aux-clercs.xml
File done : 1926_Rosny-aine-J-H_La-Femme-disparue.xml
File done : 1854_Barbey-d-Aurevilly-Jules_L-ensorcelee.xml
File done : 1910_Ivoi-Paul-d-_Millionnaire-malgre-lui-(tome-1).xml
File done : 1899_Mirbeau-Octave_Le-Jardin-des-supplices.xml
File done : 1933_Malraux-Andre_La-Condition-Humaine.xml
File done : 1931_Nizan-Paul_Aden-Arabie.xml
File done : 1880_Ulbach-Louis_Le-Crime-de-Martial-par-Louis-Ulbac

File done : 1933_Leblanc-Maurice_Victor-de-la-Brigade-mondaine.xml
File done : 1911_Apollinaire-Guillaume_Les-exploits-d-un-jeune-don-Juan.xml
File done : 1998_Rouaud-Jean_Pour-vos-cadeaux.xml
File done : 1929_Yourcenar-Marguerite_Alexis-ou-le-Traite-du-Vain-Combat-Le-Coup-de-Grace.xml
File done : 1851_Sand-George_Le-chateau-des-Desertes.xml
File done : 1913_Mauriac-François_L-enfant-charge-de-chaines.xml
File done : 1997_Modiano-Patrick_Dora-Bruder.xml
File done : 2004_Bouraoui-Nina_Poupee-Bella.xml
File done : 1947_Simon-Claude_La-corde-raide.xml
File done : 1855_Nerval-Gerard-de_Aurelia.xml
File done : 1864_Dumas-Alexandre_La-San-Felice.xml
File done : 1880_Boisgobey-Fortune-du_Le-crime-de-l-omnibus.xml
File done : 1953_Simenon-Georges_Maigret-a-peur.xml
File done : 1855_Reybaud-Louis_La-vie-de-l-employe.xml
File done : 2010_Despentes-Virginie_Apocalypse-bebe.xml
File done : 1993_Chevillard-Eric_La-nebuleuse-du-Crabe.xml
File done : 1866_Hugo-Victor_Les-travailleurs-de-la-mer.xml
Fi

File done : 1979_Echenoz-Jean_Le-meridien-de-Greenwich.xml
File done : 1843_Le-Roux-Alfred_Edouard-Aubert.xml
File done : 1975_Tournier-Michel_Les-Meteores.xml
File done : 1926_Bazin-Rene_Baltus-le-Lorrain.xml
File done : 1908_France-Anatole_L-Ile-Des-Pingouins.xml
File done : 1928_Colette_La-naissance-du-jour.xml
File done : 1951_San-Antonio_Les-souris-ont-la-peau-tendre.xml
File done : 1991_Vargas-Fred_L-homme-aux-cercles-bleus.xml
File done : 1862_Flaubert-Gustave_Salammbo.xml
File done : 1850_Dumas-Alexandre-Pere_La-femme-au-collier-de-velours.xml
File done : 1955_Laurent-Jacques_Caroline-Cherie-T.2.xml
File done : 1863_Gautier-Theophile_Le-capitaine-Fracasse.xml
File done : 1924_Hemon-Louis_Colin-Maillard.xml
File done : 1969_Mauriac-Francois_Un-adolescent-d-autrefois.xml
File done : 1904_Eekhoud-Georges_L-Autre-vue.xml
File done : 1963_Green-Julien_Partir-avant-le-jour.xml
File done : 1911_Farrere-Claude_La-maison-des-hommes-vivants.xml
File done : 1979_Ajar-Emile-Gary-Romain_L-a

File done : 1910_Cim-Albert_Le-petit-Leveille.xml
File done : 1852_Dumas-Alexandre_La-comtesse-de-Charny.xml
File done : 1913_Romains-Jules_Les-copains.xml
File done : 1930_Bernede-Arthur_L-Homme-au-masque-de-fer.xml
File done : 1986_Jonquet-Thierry_Le-Manoir-Des-Immortelles.xml
File done : 1956_Gary-Romain_Les-Racines-du-Ciel.xml
File done : 1935_Calet-Henri_La-belle-lurette.xml
File done : 2013_Toussaint-Jean-Philippe_Nue.xml
File done : 1913_Alain-Fournier_Le-grand-Meaulnes.xml
File done : 1864_Barbey-d-Aurevilly-Jules_Un-pretre-marie.xml
File done : 1972_A.D.G._La-nuit-des-grands-chiens-malades.xml
File done : 1984_Jonquet-Thierry_Mygale.xml
File done : 1983_Sarraute-Nathalie_Enfance.xml
File done : 1881_Malot-Hector_Une-femme-d-argent.xml
File done : 1890_Lemonnier-Camille_Le-possede-etude-passionnelle.xml
File done : 1918_Gide-Andre_Si-le-grain-se-meurt.xml
File done : 2012_Nothomb-Amelie_Barbe-bleue.xml
File done : 1844_Balzac-Honore-de_Splendeurs-et-miseres-des-courtisanesFC.xm

File done : 1923_Larbaud-Valery_Amants-heureux-amants.xml
File done : 1965_Perec-Georges_Les-choses.xml
File done : 1846_Balzac-Honore-de_Une-tenebreuse-affaireFC.xml
File done : 1888_Verne-Jules_Deux-ans-de-vacances.xml
File done : 1954_Reage-Pauline_Histoire-d-O.xml
File done : 1858_Dumas-Alexandre-Pere_sultanetta.xml
File done : 1895_Lermina-Jules_La-deux-fois-morte.xml
File done : 1926_Bourget-Paul_Le-danseur-mondain.xml
File done : 1972_Boileau-Narcejac_La-vie-en-miettes.xml
File done : 1920_Duhamel-Georges_Confession-de-Minuit.xml
File done : 1865_Verne-Jules_De-la-Terre-a-la-Lune.xml
File done : 2004_Grange-Jean-Christophe_La-Ligne-noire.xml
File done : 1859_Sand-George_Elle-et-Lui.xml
File done : 1998_Kundera-Milan_L-Identite.xml
File done : 1985_Tournier-Michel_La-Goutte-d-or.xml
File done : 1890_Bourget-Paul_Un-Coeur-de-femme.xml
File done : 1957_Kessel-Joseph_Le-lion.xml
File done : 1989_Simon-Claude_L-Acacia.xml
File done : 1995_Laurens-Camille_Philippe.xml
File done : 1846